# IDEA
1. Object Detection -> test_imgs의 bbox 파일 생성
    * 1-1) train / valid set split
        * train set으로 외부데이터 추가
    * 1-2) train / valid / test set을 COCO format으로 변경
    * 1-3) DetectoRS 모델 학습

2. Keypoint Detection
    * 2-1) train / valid / test set을 Keypoint Detection을 위한 형태로 변경
    * 2-2) DarkPose 모델 학습
    * 2-3) Submission file 생성

## 1-1) Train / Valid set split
* train 90% 3776장
* valid 10% 419장으로 나눴습니다.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import shutil
import tensorflow as tf
import json

from glob import glob
from PIL import Image
from tqdm import tqdm
from collections import OrderedDict

In [ ]:
# 경로 이동
os.chdir('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/')

# train, val folder 생성
root_dir = '/content/drive/My Drive/Colab/Dacon/Motion_Keypoint/'

os.makedirs(root_dir +'/train')
os.makedirs(root_dir +'/val')

# validation용 파일은 10% 비율로 random sampling
random.seed(42)
src = "train_imgs"
all_filename = os.listdir(src)
valid_filename = random.sample(all_filename, int(len(train_all) * 0.1))
train_filename = [x for x in all_filename if x not in valid_filename]

print(len(train_filename), len(valid_filename))

train_filename = [src+'/'+ name for name in train_filename]
valid_filename = [src+'/' + name for name in valid_filename]

print('Total images: ', len(all_filename))
print('Training: ', len(train_filename))
print('Validation: ', len(valid_filename))

# copy & paste images
for name in tqdm(train_filename):
    shutil.copy(name, 'train')

for name in tqdm(valid_filename):
    shutil.copy(name, 'val')

In [ ]:
# DACON 제공 train_imgs.csv read
train =pd.read_csv(root_dir + 'train_df.csv')

In [ ]:
# 각각의 train, valid 이미지들의 정보만을 담고 있는 DataFrame 생성
train_df = train[train['image'].isin(train_filename)]
train_df.reset_index(inplace=True, drop=True)

valid_df = train[train['image'].isin(valid_filename)]
valid_df.reset_index(inplace=True, drop=True)

train_df.to_csv('train.csv', index=False)
valid_df.to_csv('valid.csv', index=False)

In [ ]:
# Colab상 경로 재설정
%cd /content

### Train set으로 외부데이터 추가
(AI Hub) 사람 동작 영상 AI 데이터 28번 웨이트트레이닝 - 이미지에서 약 300장 추출
* Supervisely를 통해 bbox 작업
* 이미지 출처(https://aihub.or.kr/aidata/138/download)
* 사용한 이미지와 작성한 bbox는 share folder에 첨부

Object Detection에서 몇몇 사진의 경우, 사람이 아닌 의자를 탐지한 사진을 발견,추가 데이터 학습으로 사람을 추출하게끔 작업

In [ ]:
# 추가 이미지 파일은 share_folder의 추가데이터/additional_data 폴더에 위치
# additional_ann은 Supervisely에서 작업한 train용 bbox 정보가 담긴 json 파일들의 폴더
ann_path = './data/Motion_Keypoint/additional_ann/'
file_list = glob(ann_path + '*.json')
file_list.sort()
len(file_list)  # 308

In [ ]:
json_file = file_list[0].split('/')[-1]
img_file = json_file.split('.')[0] + '.jpg'
print(json_file)
print(img_file)

In [ ]:
# 추가 데이터를 집어넣을 빈 데이터프레임 생성
additional_df = pd.DataFrame(columns=['images', 'xmin', 'ymin', 'xmax', 'ymax'])

# 빈 데이터프레임에 image 파일명 채우기
file_name_list = []
for i in range(len(file_list)):
    file_name = file_list[i].split('/')[-1]
    file_name = file_name.split('.json')[0]
    file_name_list.append(file_name)
    
additional_df['images'] = file_name_list

In [ ]:
# Supervisely에서 작업한 bbox는 xmin, ymin, xmax, ymax 형태
# bbox들을 additional_df에 file_list 순서에 맞춰 넣어줌

for i in range(len(file_list)):

    with open(file_list[i], 'r') as f:
        json_data = json.load(f)
    
    #json 파일의 points 추출
    points = json_data['objects'][0]['points']['exterior']
    xmin = points[0][0]
    ymin = points[0][1]
    xmax = points[1][0]
    ymax = points[1][1]

    coordinates = []
    coordinates.append(xmin)
    coordinates.append(ymin)
    coordinates.append(xmax)
    coordinates.append(ymax)
    
    additional_df.iloc[i,1:] = coordinates

In [ ]:
# additional_df 저장
additional_df.to_csv('./data/Motion_Keypoint/additional_df.csv', index=False)

In [ ]:
# train 데이터와 추가 데이터를 하나의 폴더안에 합쳐줍니다.
%cd /content
os.chdir('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/')

src1 = "train"
src2 = "additional_data"

os.makedirs(root_dir +'/train_add')
train_filename = os.listdir(src1)
add_filename = os.listdir(src2)

train_filename = [src+'/'+ name for name in train_filename]
add_filename = [src+'/'+ name for name in add_filename]

# copy & paste images
for name in tqdm(train_filename):
    shutil.copy(name, 'train_add')

for name in tqdm(add_filename):
    shutil.copy(name, 'train_add')

%cd /content

## 1-2) train / valid / test set을 COCO format으로 변경

train용 파일, 추가 학습용 파일 변경

In [ ]:
# 들어온 image를 json 파일형식으로 변환하는 코드
# 폴더에서 jpg 형식의 파일들을 모두 읽은 다음 sort
image_paths = glob(os.path.join('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train','*.jpg'))
image_paths.sort()
len(image_paths)

In [ ]:
#json 형식으로 담을 dict 생성
file_data = OrderedDict()
file_data['images'] = []

# for문으로 mmdetection에서 사용할 file_data 형식으로 변형
file_id = 0
for i in tqdm(range(len(image_paths))):
    img = Image.open(image_paths[i])
    filename = image_paths[i].split('/')[-1]
    img_dic = {'file_name': filename, 'height': img.size[1], 'width': img.size[0], 'id': file_id}
    file_data['images'].append(img_dic)
    file_id += 1

# 3776이어야 함
print(file_id)

In [ ]:
# Object_Detection용 추가 데이터
add_image_paths = glob(os.path.join('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/additional_data','*.jpg'))
add_image_paths.sort()
print(len(add_image_paths))
print(file_id)

# 앞선 셀의 마지막 file_id는 img_dic에 들어가지 않았으므로
# 계속 이어 받아도 무방
for j in tqdm(range(len(add_image_paths))):
    imge = Image.open(add_image_paths[j])
    filenmae = add_image_paths[j].split('/')[-1]
    img_dic = {'file_name': filename, 'height': img.size[1], 'width': img.size[0], 'id': file_id}
    file_data['images'].append(img_dic)
    file_id += 1

In [ ]:
# bbox 계산을 위해 csv 파일 read
# df = pd.read_csv('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train(0402).csv')
df = df.sort_values(by=['image'], axis=0)
df = df.reset_index(drop=True)
print(len(df))
df.tail(3)

In [ ]:
# for문으로 mmdetection에서 사용할 file_data 형식으로 변형
# file_data['annotations']은 -> {'segmentation': [], 'area':, 'category_id': label, 'bbox': [], image_id: file_id, 'iscrowd': 0, 'id': file_id} 형식으로 들어가야 함
# COCO format에서 bbox는 [xmin, ymin, w, h] 형식임 -> [xmin, ymin, xmax-xmin, ymax-ymin]
# image_paths의 경우 sort를 했고, df 또한 이름순으로 이미 정렬되어 있으므로 images의 id와 annotation의 image_id는 자연스레 mapping

file_data['annotations'] = []

image_id = 0
annotation_id = 0
for k in tqdm(range(len(image_paths))):
    filename = df.iloc[k, 0]
    xmin, ymin = min(df.iloc[k, 1::2]), min(df.iloc[k, 2::2])
    xmax, ymax = max(df.iloc[k, 1::2]), max(df.iloc[k, 2::2])
    w = xmax - xmin
    h = ymax - ymin
    bbox = [xmin, ymin, w, h]
    area = float(w * h)
    seg_dic = {'segmentation': [], 'area': area, 'category_id': 1, 'bbox': bbox, 'image_id': image_id, 'iscrowd': 0, 'id': annotation_id}
    file_data['annotations'].append(seg_dic)
    image_id += 1
    annotation_id += 1

In [ ]:
additional_df = pd.read_csv('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/additional_df.csv')
additional_df = additional_df.sort_values(by=['images'], axis=0)
additional_df = additional_df.reset_index(drop=True)
print(len(additional_df))
additional_df.head(2)

In [ ]:
# 둘다 3776
print(image_id)
print(annotation_id)

# 앞선 셀의 마지막 image_id, annotation_id는 seg_dic 들어가지 않았으므로
# 계속 이어 받아도 무방
for l in tqdm(range(len(add_image_paths))):
    xmin, ymin = additional_df.loc[l, 'xmin'], additional_df.loc[l, 'ymin']
    xmax, ymax = additional_df.loc[l, 'xmax'], additional_df.loc[l, 'ymax']
    xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
    w = xmax - xmin
    h = ymax - ymin
    bbox = [xmin, ymin, w, h]
    area = float(w * h)
    seg_dic = {'segmentation': [], 'area': area, 'category_id': 1, 'bbox': bbox, 'image_id': image_id, 'iscrowd': 0, 'id': annotation_id}
    file_data['annotations'].append(seg_dic)
    image_id += 1
    annotation_id += 1

In [ ]:
# mmdetection에서 사용할 categories 형식도 추가
# 분류값은 사람뿐이므로 person만 추가
file_data['categories'] = [{"id": 1, "name": "person"}]

In [ ]:
# json 파일로 저장
with open('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train_mmdetection_add.json', 'w', encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False)

validation용 파일 변환

In [ ]:
image_paths = glob(os.path.join('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/val','*.jpg'))
image_paths.sort()
len(image_paths)

In [ ]:
#json 형식으로 담을 dict 생성
file_data = OrderedDict()
file_data['images'] = []

# for문으로 mmdetection에서 사용할 file_data 형식으로 변형
file_id = 0
for i in tqdm(range(len(image_paths))):
    img = Image.open(image_paths[i])
    filename = image_paths[i].split('/')[-1]
    img_dic = {'file_name': filename, 'height': img.size[1], 'width': img.size[0], 'id': file_id}
    file_data['images'].append(img_dic)
    file_id += 1

In [ ]:
# bbox 계산을 위해 csv 파일 read
df = pd.read_csv('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/valid(0402).csv')
df = df.sort_values(by=['image'], axis=0)
df = df.reset_index(drop=True)
print(len(df))
df.head(3)

In [ ]:
# for문으로 mmdetection에서 사용할 file_data 형식으로 변형
# file_data['annotations']은 -> {'segmentation': [], 'area':, 'category_id': label, 'bbox': [], image_id: file_id, 'iscrowd': 0, 'id': file_id} 형식으로 들어가야 함
# COCO format에서 bbox는 [xmin, ymin, w, h] 형식임 -> [xmin, ymin, xmax-xmin, ymax-ymin]
# image_paths의 경우 sort를 했고, df 또한 이름순으로 이미 정렬되어 있으므로 images의 id와 annotation의 image_id는 자연스레 mapping

file_data['annotations'] = []

image_id = 0
annotation_id = 0
for k in tqdm(range(len(image_paths))):
    filename = df.iloc[k, 0]
    xmin, ymin = min(df.iloc[k, 1::2]), min(df.iloc[k, 2::2])
    xmax, ymax = max(df.iloc[k, 1::2]), max(df.iloc[k, 2::2])
    w = xmax - xmin
    h = ymax - ymin
    bbox = [xmin, ymin, w, h]
    area = float(w * h)
    seg_dic = {'segmentation': [], 'area': area, 'category_id': 1, 'bbox': bbox, 'image_id': image_id, 'iscrowd': 0, 'id': annotation_id}
    file_data['annotations'].append(seg_dic)
    image_id += 1
    annotation_id += 1

In [ ]:
# mmdetection에서 사용할 categories 형식도 추가
# 분류값은 사람뿐이므로 person만 추가
file_data['categories'] = [{"id": 1, "name": "person"}]

In [ ]:
# json 파일로 저장
with open('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/valid_mmdetection.json', 'w', encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False)

Test용 파일 변환

In [ ]:
image_paths = glob(os.path.join('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/test_imgs','*.jpg'))
image_paths.sort()
len(image_paths)

In [ ]:
#train_json 형식으로 담을 dict 생성
file_data = OrderedDict()
file_data['images'] = []

# for문으로 mmdetection에서 사용할 file_data 형식으로 변형
file_id = 0
for i in tqdm(range(len(image_paths))):
    img = Image.open(image_paths[i])
    filename = image_paths[i].split('/')[-1]
    img_dic = {'file_name': filename, 'height': img.size[1], 'width': img.size[0], 'id': file_id}
    file_data['images'].append(img_dic)
    file_id += 1

In [ ]:
# mmdetection에서 사용할 categories 형식도 추가
# 분류값은 사람뿐이므로 person만 추가
file_data['categories'] = [{"id": 1, "name": "person"}]

In [ ]:
# json 파일로 저장
with open('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/test_mmdetection.json', 'w', encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False)

## 1-3) DetectoRS 모델 학습(MMdetection)
* ResNeXt-101-64x4d, multi-scale을 시도했으나 의의로 Resnet-50의 성과가 더 좋게 나왔음.

In [ ]:
# 이미 사전에 특정 위치에 설치한 관계로 경로 이동
# Colab은 도커형식으로 특정 위치로 이동 후 mmdetection을 clone하지 않으면
# 해당 폴더는 연결이 끊기는 순간 사라짐
%cd drive/MyDrive

# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# Colab 상에서의 mmdetection 환경설정
# Colab Pro에서는 CUDA 11.0 버젼이지만 mmdetection installation docs 따라 설치하겠습니다.

# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.2.7

# Install mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

In [ ]:
# 학습 진행
# 학습을 위한 configs 파일들은 share folder에 첨부. mmdetection folder 내 같은 경로에 넣어주면 됩니다.
# Colab에서 사용시 /usr/local/lib/python3.7/dist-packages/mmcv/runner/epoch_based_runner.py 해당 경로의
# 135번째 줄 save_checkpoint함수의 create_symlink=False로 바꿔줘야 latest.pth 파일 오류 생성 x

    # img_prefix=data_root + 'train_add'에서 train_add는 train image와 추가데이터 image가 함께 있는 폴더입니다.
# mmdet/datasets/coco.py 역시 수정 파일로 share folder에 첨부

!python tools/train.py configs/detectors/detectors_cascade_rcnn_r50_1x_coco_0329.py

In [ ]:
# test 진행
!python tools/test.py configs/detectors/detectors_cascade_rcnn_r50_1x_coco_0329.py work_dirs/detectors_cascade_rcnn_r50_1x_coco_0329/epoch_20.pth --format-only --eval-options "jsonfile_prefix=./detectors_epoch_20"

## 2-1) train / valid / test set을 Keypoint Detection을 위한 형태로 변경
annotation을 지정해 줘야 하기 때문에 변경을 한번 더 수행합니다.

train set 변환

In [ ]:
# mmdetection용 json파일 생성 방식과 거의 동일합니다.
# Keypoint Detection에는 추가 데이터를 사용하지 않습니다.
%cd /content

os.chdir('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint')
image_paths = glob(os.path.join('train','*.jpg'))
image_paths.sort()
len(image_paths)

In [ ]:
# COCO images 처리
file_data = OrderedDict()
file_data['images'] = []

id = 0
for i in tqdm(range(len(image_paths))):
    img = Image.open(image_paths[i])
    filename = image_paths[i].split('/')[-1]
    file_id = filename.split('.')[0]
    img_dic = {'file_name': filename, 'height': img.size[1], 'width': img.size[0], 'id': id}
    file_data['images'].append(img_dic)
    id += 1

In [ ]:
# COCO annotations 처리
train = pd.read_csv('./train.csv')
train.head(3)

In [ ]:
# annotations의 전체 keypoints 생성
# COCO format은 keypoint 지정에 v축을 요구
# 사진을 다 확인할 수 없는 관계로 v를 2로 설정
# 사진을 다 확인할 수 없는 관계로 v를 2로 설정

nose_xy = train.iloc[:, 1:3].values
properties = np.array([[2]]*len(train))
keypoints = np.concatenate((nose_xy, properties), axis=1)

for i in tqdm(range(23)):
    xy = train.iloc[:, 3+(2*i):5+(2*i)].values
    v = np.array([[2]]*len(train))
    keypoint = np.concatenate((xy,v), axis=1)
    keypoints = np.concatenate((keypoints, keypoint), axis=1)

In [ ]:
file_data['annotations'] = []
id = 0

## COCO format은 keypoint 지정에 v축을 요구
## 사진을 다 확인할 수 없는 관계로 v를 2로 설정
for i in tqdm(range(len(train))):
    file_name = train.loc[i].image
    file_id = file_name.split('.')[0]
    key = keypoints[i]
    xmin = train.iloc[i, 1::2].values.min()
    xmax = train.iloc[i, 1::2].values.max()
    w = xmax - xmin
    ymin = train.iloc[i, 2::2].values.min()
    ymax = train.iloc[i, 2::2].values.max()
    h = ymax - ymin
    bbox = [xmin, ymin, w,h]
    area = w * h
    annotation_dic = {'segmentation': [], 'keypoints': key.tolist(), 'num_keypoints': 24, 'area': area, 'iscrowd': 0, 'image_id': id, 'bbox': bbox, 'category_id': 1, 'id': id}
    file_data['annotations'].append(annotation_dic)
    id += 1

In [ ]:
file_data['categories'] = [{'id': 1, 
                            'name': 'person',
                            "keypoints": ["nose","left_eye","right_eye","left_ear","right_ear",
                                          "left_shoulder","right_shoulder","left_elbow","right_elbow",
                                          "left_wrist","right_wrist","left_hip","right_hip",
                                          "left_knee","right_knee","left_ankle","right_ankle",
                                          "neck", "left_palm", "right_palm", "spine2(back)", "spine1(back)", "left_instep", "right_instep"],
                            "skeleton": [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13],[18,6],[18,7],
                                         [6,8],[7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7],
                                         [1,18], [18,21], [21,22], [22,12], [22,13], [23,16], [24,17]]}]

In [ ]:
with open('./train_ann.json', 'w', encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False)

valid set 변환

In [ ]:
# mmdetection용 json파일 생성 방식과 거의 동일합니다.
# Keypoint Detection에는 추가 데이터를 사용하지 않습니다.
%cd /content

os.chdir('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint')
image_paths = glob(os.path.join('val','*.jpg'))
image_paths.sort()
len(image_paths)

In [ ]:
# COCO images 처리
file_data = OrderedDict()
file_data['images'] = []

id = 0
for i in tqdm(range(len(image_paths))):
    img = Image.open(image_paths[i])
    filename = image_paths[i].split('/')[-1]
    file_id = filename.split('.')[0]
    img_dic = {'file_name': filename, 'height': img.size[1], 'width': img.size[0], 'id': id}
    file_data['images'].append(img_dic)
    id += 1

In [ ]:
# COCO annotations 처리
valid = pd.read_csv('./valid.csv')
valid.head(3)

In [ ]:
# annotations의 전체 keypoints 생성
# COCO format은 keypoint 지정에 v축을 요구
# 사진을 다 확인할 수 없는 관계로 v를 2로 설정
# 사진을 다 확인할 수 없는 관계로 v를 2로 설정

nose_xy = valid.iloc[:, 1:3].values
properties = np.array([[2]]*len(valid))
keypoints = np.concatenate((nose_xy, properties), axis=1)

for i in tqdm(range(23)):
    xy = valid.iloc[:, 3+(2*i):5+(2*i)].values
    v = np.array([[2]]*len(valid))
    keypoint = np.concatenate((xy,v), axis=1)
    keypoints = np.concatenate((keypoints, keypoint), axis=1)

In [ ]:
file_data['annotations'] = []
id = 0

## COCO format은 keypoint 지정에 v축을 요구
## 사진을 다 확인할 수 없는 관계로 v를 2로 설정
for i in tqdm(range(len(valid))):
    file_name = valid.loc[i].image
    file_id = file_name.split('.')[0]
    key = keypoints[i]
    xmin = valid.iloc[i, 1::2].values.min()
    xmax = valid.iloc[i, 1::2].values.max()
    w = xmax - xmin
    ymin = valid.iloc[i, 2::2].values.min()
    ymax = valid.iloc[i, 2::2].values.max()
    h = ymax - ymin
    bbox = [xmin, ymin, w,h]
    area = w * h
    annotation_dic = {'segmentation': [], 'keypoints': key.tolist(), 'num_keypoints': 24, 'area': area, 'iscrowd': 0, 'image_id': id, 'bbox': bbox, 'category_id': 1, 'id': id}
    file_data['annotations'].append(annotation_dic)
    id += 1

In [ ]:
file_data['categories'] = [{'id': 1, 
                            'name': 'person',
                            "keypoints": ["nose","left_eye","right_eye","left_ear","right_ear",
                                          "left_shoulder","right_shoulder","left_elbow","right_elbow",
                                          "left_wrist","right_wrist","left_hip","right_hip",
                                          "left_knee","right_knee","left_ankle","right_ankle",
                                          "neck", "left_palm", "right_palm", "spine2(back)", "spine1(back)", "left_instep", "right_instep"],
                            "skeleton": [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13],[18,6],[18,7],
                                         [6,8],[7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7],
                                         [1,18], [18,21], [21,22], [22,12], [22,13], [23,16], [24,17]]}]

In [ ]:
with open('./valid_ann.json', 'w', encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False)

test set 변환

In [ ]:
# MMpose의 ann_file은 id를 file_name으로 읽기 때문에 위에서 만든 test_mmdetection.json의
# id값만 file_name으로 맞춰서 변경

file_path = "/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/test_mmdetection.json"
with open(file_path, 'r') as f:
    test_mmdetection = json.load(f)

for images in test_mmdetection['images']:
    images['id'] = images['file_name']

with open('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/test_0331.json', 'w', encoding='utf-8') as make_file:
    json.dump(test_mmdetection, make_file, ensure_ascii=False)

추가적으로 data_cfg 중 bbox_file에 넣어줄 형식도 필요합니다.

train에 쓰일 data_cfg

In [ ]:
# train과 개수만 맞으면 되기 때문에 위에서 만든 train_ann을 임의로 불러옵니다.
# "image_id": "001-1-1-01-Z17_A-0000001.jpg" 아래와 같이 파일명으로 지정해야 bbox_file을 읽는데 오류 x

file_path = "/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train_ann.json"
with open(file_path, 'r') as f:
    data = json.load(f)

file_data = []
# train의 경우, score값은 1로 지정
for i in tqdm(range(len(data['images']))):
    dic = {'bbox': data['annotations'][i]['bbox'], 'category_id': 1, 'image_id': data['annotations'][i]['image_id'], 'score': 1.0}
    file_data.append(dic)

for i in tqdm(range(len(file_data))):
    file_data[i]['image_id'] = file_data[i]['image_id'] + '.jpg'

In [ ]:
# 파일명
with open('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train_mmpose_bbox.json', 'w', encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False)

test에 쓰일 data_cfg

In [ ]:
# test에 쓰일 data_cfg는 Object Detection으로 생성된 bbox로 bbox_file로 생성
# "image_id": "001-1-1-01-Z17_A-0000001.jpg" 아래와 같이 파일명으로 지정해야 bbox_file을 읽는데 오류 x

file_path = "/content/drive/MyDrive/mmdetection/detectors_epoch_20.bbox.json"
with open(file_path, 'r') as f:
    bbox = json.load(f)

for images in bbox['bbox']:
    images['id'] = images['file_name']

with open('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/train_mmpose_bbox.json', 'w', encoding='utf-8') as make_file:
    json.dump(bbox, make_file, ensure_ascii=False)

## 2-2) DarkPose(MMpose)

In [ ]:
# 이미 사전에 특정 위치에 설치한 관계로 경로 이동
# Colab은 도커형식으로 특정 위치로 이동 후 mmpose를 clone하지 않으면
# 해당 폴더는 연결이 끊기는 순간 사라짐
%cd /content
%cd drive/MyDrive

# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# # install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.2.7

# # Install mmdetection
# !rm -rf mmpose
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
!pip install -r requirements.txt
!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

In [ ]:
# mmpose의 경우, 
# requirements.txt의 numpy 1.19.5버전이 오류를 발생시키는 관계로
# numpy를 1.20.0 버전으로 업데이트
!pip install numpy==1.20.0

In [ ]:
# Keypoint Detection 학습
# 학습을 위한 configs 파일들은 share folder에 첨부. mmpose folder 내 같은 경로에 넣어주면 됩니다.
# mmdet/datasets/datasets/top_down/topdown_coco_dataset.py 역시 수정 파일로 share folder에 첨부
# Colab에서 사용시 /usr/local/lib/python3.7/dist-packages/mmcv/runner/epoch_based_runner.py 해당 경로의
# 135번째 줄 save_checkpoint함수의 create_symlink=False로 바꿔줘야 latest.pth 파일 오류 생성 x

# --work-dir 경로에 학습한 모델들이 저장
!python tools/train.py configs/top_down/darkpose/coco/hrnet_w48_coco_384x288_dark.py --work-dir work_dirs/darkpose

In [ ]:
# test할 경우에는
# hrnet_w48_coco_384x288_dark.py의 data_cfg의 'bbox_file'을 위에서 만든 train_mmpose_bbox.json으로 바꿔줘야 함
# share folder에 올린 hrnet_w48_coco_384x288_dark.py에 주석처리 해둠
# -out 명령어로 파일명 지정
!python tools/test.py configs/top_down/darkpose/coco/hrnet_w48_coco_384x288_dark.py work_dirs/darkpose/epoch_205.pth --out 'darkpose.json'

## Make Submission

In [ ]:
# DakrPose가 예측한 json 파일을 불러옵니다.

file_path = "./darkpose.json"
with open(file_path, 'r') as f:
    result = json.load(f)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/sample_submission.csv')

In [ ]:
pred = []
for output in tqdm(result):
    row = []
    image_name = output['image_paths'][0].split('/')[-1]
    row.append(image_name)
    for i in range(24):
        row.append(output['preds'][0][i][0])
        row.append(output['preds'][0][i][1])
    pred.append(row)

In [ ]:
column = submission.columns
submission = pd.DataFrame.from_records(pred, columns=column)

In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab/Dacon/Motion_Keypoint/darkpose.csv', index=False)